In [12]:
# imports
import pandas as pd
import requests
import os
import numpy as np

In [13]:
#Necessary functions:

def get_json(url, headers_dict, params_dict):
    '''Prints status code of url and gets json file'''
    res = requests.get(url, headers = headers_dict, params = params_dict)
    print(f"The status code is {res.status_code}.")
    return res.json()


# Foursquare

Send a request to Foursquare with a small radius (1000m) with the location of your choice

In [14]:
foursquare_api = os.environ["FOURSQUARE_API_KEY"]

url = 'https://api.foursquare.com/v3/places/search'
headers_dict = {
            'Accept': 'application/JSON',
             'Authorization': foursquare_api}

location = '43.67,-79.43'
rad = 1000 #Radius from location in metres
v_date = '20220901' # version data required for Foursquare
num_results = 50 # Get this many results
params_dict = {'ll': location, 'radius': rad,  'v': v_date, 'limit':num_results}

fsq_results = get_json(url, headers_dict, params_dict)


The status code is 200.


Parse through the response to get the POI details you want (rating, name, location, etc)

In [16]:
def json_info(json_file, dict_key, website):
    '''Creates dictionary based on input json file, taken from an API
        Works with Yelp, Foursquare, so far
        dict_key is a site-specific way information is stored (taken from raw data'''
    json_dict = {}
    for num, venue in enumerate(json_file[dict_key]):
        venue_dict = {}
        venue_dict['name'] = venue['name'] #Name venue
        
        if (website == "Yelp"): #Janky solution but idc
            venue_dict['rating'] = venue['rating']
        #Can have multiple categories, so extract them to a list 
        categories_dict = {key:value for key, value in venue['categories'][0].items()} #Adds various info from original location key
        location_dict = {key:value for key, value in venue['location'].items()} #Adds various info from original location key
        venue_dict.update(categories_dict) #append dictionaries
        venue_dict.update(location_dict) #append dictionaries

        #Cleaning up formatting that is Foursquare specific 
        if (website == 'Foursquare'):
            venue_dict['category'] = categories_dict['name']
            venue_dict['fsq_id'] = venue['fsq_id']
            venue_dict['name'] = venue['name'] #Name venue again, bc Foursquare overwrites the name from the 'categories' menu
            del venue_dict['icon']
            ###
        if (website == 'Yelp'):
            venue_dict['display_address'] = venue_dict['display_address'][0]

        json_dict[f'venue_{num+1}'] = venue_dict
    
    return json_dict        



In [17]:
#So passing a string directly toa  function is bad. But save it to a variable and pass the variable is good
location_key_name = 'results'
website = 'Foursquare'
fsq_data = json_info(fsq_results, location_key_name, website )


Put your parsed results into a DataFrame

In [57]:
fsq_df = pd.DataFrame.from_dict(fsq_data, orient = 'index')

##### Temporary solution: just delete the stupid column
fsq_df.drop('neighborhood', axis=1, inplace=True)
# fsq_df['street'] = fsq_df['address'].map(lambda x: x.lstrip('0123456789 '))
# fsq_df['neighborhood'] = fsq_df['neighborhood'][0] 
# fsq_df['neighborhood'].astype('|S')

####Uncomment this to see dataframe
# fsq_df

In [19]:
#Now we need to search for Foursquare 'tips' or whatever the hell
# url = f'https://api.foursquare.com/v3/places/{venue_id}/tips'
url = 'https://api.foursquare.com/v3/places/568333d5498e400ae0f1a732/tips'
headers_dict = {
            'Accept': 'application/JSON',
             'Authorization': foursquare_api}

params_dict = {'limit': 50}

fsq_tips_results = get_json(url, headers_dict, params_dict)

def json_fsq_tips(json_file) :
    '''Creates dictionary based on Foursquare json file, taken from tips API'''
    json_dict = {}
    for num, venue in enumerate(json_file):
        
        venue_dict = {key:value for key, value in venue.items()} #Adds various info from original location key
       
        json_dict[f'review_{num+1}'] = venue_dict
    
    return json_dict        

fsq_tips_json = json_fsq_tips(fsq_tips_results)

tips_df = pd.DataFrame.from_dict(fsq_tips_json, orient = 'index')

# tips_df.loc['review_5', 'text']




The status code is 200.


# Yelp

Send a request with the same location paramaters (location, radius, etc)

In [20]:
yelp_api = os.environ["YELP_API_KEY"]
url = 'https://api.yelp.com/v3/businesses/search' 
headers_dict = {
            'Accept': 'application/JSON',
             'Authorization': f'Bearer {yelp_api}'
             }

lat = '43.67' 
long = '-79.43'
rad = 2000 #Radius from location in metres
v_date = '20220901' # version data required for Foursquare
num_results = 50 # Number of search results to display
params_dict = {'latitude': lat, 'longitude': long, 'radius': rad, 'limit': num_results}

yelp_results = get_json(url, headers_dict, params_dict)

The status code is 200.


Parse through your result and get POI details

In [58]:
website = 'Yelp'
location_key_name = 'businesses'
yelp_data = json_info(yelp_results, location_key_name, website)
# print(yelp_data['venue_1'])


Put your parsed results into a DataFrame

In [60]:
yelp_df = pd.DataFrame.from_dict(yelp_data, orient = 'index')

#Find street the business is on
yelp_df['street'] = yelp_df['display_address'].map(lambda x: x.lstrip('0123456789 '))

####getting rid of nan in neighborhood?

#df2 = df.astype(object).where(pd.notnull(df), None)

##Uncomment this to see dataframe
# yelp_df

# Google (stretch)

Use the same process as the first two APIs

In [63]:
google_api = os.environ["GOOGLE_API_KEY"]
url = 'https://maps.googleapis.com/maps/api/directions' 
headers_dict = {
            'Accept': 'application/JSON'}

# lat = '43.67' 
# long = '-79.43'
# rad = 1000 #Radius from location in metres
# v_date = '20220901' # version data required for Foursquare
# num_results = 50 # Number of search results to display
# params_dict = {'latitude': lat, 'longitude': long, 'radius': rad, 'limit': num_results}


params_dict = {'key': google_api}

# yelp_results = get_json(url, headers_dict, params_dict)

# Database

Put all your results in an SQLite3 database (remember, SQLite stores its databases as files in your local machine - make sure to create your database in your project's data/ directory!)

In [46]:
import sqlite3
from sqlite3 import Error

def create_connection(path):
    connection = None

    try:
        connection = sqlite3.connect(path)
        print("Connection to SQLite DB successful")
    except Error as e:
        print(f"The error '{e}' occured.")

    return connection

def execute_query(connection, query):
    cursor = connection.cursor()
    try:
        cursor.execute(query)
        connection.commit()
        print("Query executed successfully")
    except Error as e:
        print(f"The error '{e}' occurred")

def execute_read_query(connection, query):
    cursor = connection.cursor()
    result = None
    try:
        cursor.execute(query)
        result = cursor.fetchall()
        return result
    except Error as e:
        print(f"The error '{e}' occurred")


connection = create_connection("poi.db")


##Do this to delete tables so we can start fresh (if we need to)
drop_table = 'DROP TABLE fsq_sql'
execute_query(connection, drop_table)
drop_table = 'DROP TABLE yelp_sql'
execute_query(connection, drop_table)

fsq_df.to_sql(name='fsq_sql', con = connection)
yelp_df.to_sql(name='yelp_sql', con = connection)

##Add tables to sql, then merge

Connection to SQLite DB successful
The error 'no such table: fsq_sql' occurred
The error 'no such table: yelp_sql' occurred


50

Get the top 10 restaurants according to their rating

In [47]:
select = 'SELECT name, rating FROM yelp_sql \
          ORDER BY rating DESC \
          LIMIT 10;'

tempy = execute_read_query(connection, select)
for temp in tempy:
    print(temp)

('La Bella Managua', 4.5)
('Hodo Kwaja', 4.5)
('Sugo', 4.5)
('Famiglia Baldassarre', 4.5)
('Ramen Isshin', 4.5)
('Rustle & Still Café', 4.5)
('Bar Ape', 4.5)
('Good Behaviour', 4.5)
("Kevin's Taiyaki", 4.5)
('Jin Dal Lae', 4.5)


Find type of local restaurants

In [48]:

select = 'SELECT category, COUNT(category) FROM fsq_sql\
          GROUP BY category \
          ORDER BY COUNT(category) DESC \
          LIMIT 10;'

tempy = execute_read_query(connection, select)
for temp in tempy:
    print(temp)

('Bakery', 5)
('Bar', 4)
('Tattoo Parlor', 3)
('Park', 2)
('Laundry Service', 2)
('Interior Designer', 2)
('Grocery Store / Supermarket', 2)
('French Restaurant', 2)
('Café', 2)
('Storage Facility', 1)


In [33]:
#Find types of local businesses
select = 'SELECT alias, COUNT(alias) FROM yelp_sql \
          GROUP BY alias \
          ORDER BY COUNT(alias) DESC \
          LIMIT 10;'

tempy = execute_read_query(connection, select)
for temp in tempy:
    print(temp)

('korean', 8)
('italian', 4)
('indpak', 3)
('icecream', 3)
('desserts', 3)
('thai', 2)
('newcanadian', 2)
('mexican', 2)
('japanese', 2)
('bakeries', 2)


In [50]:
##Find most common street of businesses 
select = 'SELECT street, COUNT(street) FROM yelp_sql \
          GROUP BY street \
          ORDER BY COUNT(street) DESC \
          LIMIT 10;'

tempy = execute_read_query(connection, select)
for temp in tempy:
    print(temp)

('Bloor Street W', 24)
('College Street', 5)
('St. Clair Avenue W', 3)
('St Clair Avenue W', 3)
('Geary Avenue', 2)
('Westmoreland Avenue N', 1)
('W Bloor Street', 1)
('Vaughan Road', 1)
('Saint Clair Avenue W', 1)
('Rushton Rd', 1)


In [56]:
#Finding businesses that are in both Foursquare and Yelp queries

execute_read_query(connection, select)
#Find types of local businesses
select = 'SELECT yelp_sql.name FROM yelp_sql \
          INNER JOIN fsq_sql ON yelp_sql.name = fsq_sql.name;'

tempy = execute_read_query(connection, select)
for temp in tempy:
    print(temp)


The error 'ambiguous column name: name' occurred
('Bakerbots Baking',)
('La Bella Managua',)
('Famiglia Baldassarre',)
('Good Behaviour',)
('Actinolite Restaurant',)
('Civil Liberties',)
('Parallel',)
('Tallboys Craft Beer House',)


# Travelling Salesman Problem (stretch)

If you have time, follow the steps in the [ortools tutorial](https://developers.google.com/optimization/routing/tsp) using Google's [Directions API](https://developers.google.com/maps/documentation/directions/start).